In [1]:
from util import kafkaProducer

: 

: 

In [2]:
csv_path="data/camera_event_a.csv"
#ip address is the zookeeper & port to the kafka port
kafka_server="172.17.0.1:9092"
producer_id="producer_a"
topic="camera_event_a"
batch_interval=5

producer = kafkaProducer(csv_path, kafka_server, producer_id, topic, batch_interval)
producer.produce_batches()

: 